# Class Imbalance

간단하게 설명해서, 이미지 하나당 40000~50000개 정도를 추천하는데, 이중에 object가 있을거라 추정하는 위치는 매우 적습니다. <br>
문제는 다음과 같습니다.

1. 대부분의 위치들은 **easy negatives** (background라고 판단되는 확률값이 0.5 이상인 녀석들)이며, easy negatives는 실제 학습에 크게 도움을 주지 않습니다. <br>easy negatives들 자체의 loss값은 작지만 이런 값들 자체의 양이 많고, 모두 합친 loss값은 rare class에서 나온 loss값을 압도해버립니다. 


> <span style="color:#999999"> 페이퍼에서 **easy** 라는 말이 많이 쓰이는데.. 해당 class (background 또는 foreground)를 0.5 확률이상으로 예측한 것을 말합니다. <br>
> 즉 높은 확률값으로 배경이다 또는 객체이다를 예측한 위치를 말합니다.</span>


# Focal Loss

## Cross Entropy

Focal Loss은 one-stage object detection 시나리오를 다루기 위해서 디자인 되었으며 학습도중의 foreground 그리고 background classes의 극단적인 class imbalance 문제를 해결합니다.

먼저 binary cross entropy를 시작으로 해서 focal loss를 정의하도록 합니다.

$$ \text{CE}(p, y) = 
\begin{cases} 
- \log(p) & \text{if}\ y = 1 \\ 
- \log(1-p) & \text{otherwise} \\
\end{cases} $$

* $ y \in \{ \pm 1 \} $  :  y 는  1 또는 -1 이며, ground-truth class를 나타낸다.
* $ p \in [ 0 , 1 ]  $ : p는 0 ~ 1 사이의 값이며, 모델이 추정한 확률값 (estimated probability) 입니다 (1에 가까울수록 ground-truth class)

notation을 줄여서 쓰기 위해서 $ P_t $ 를 다음과 같이 정의 합니다.

$$ P_t = 
\begin{cases} 
p & \text{if } y = 1 \\
1 - p & \text{otherwise}
\end{cases} $$

$$ CE(p, y) = CE(p_t) = -log(p_t) $$

## Balanced Cross Entropy

class imbalance문제를 해결하는 가장 일반적인 방법은 weighting factor $ \alpha \in [0, 1] $ 를 class 1에 그리고 $ 1 - \alpha $ 를 class -1 에 적용하는 것 입니다.

$$ \text{CE}(p_t) = - \alpha_t \log(p_t) $$

## Focal Loss Definition

페이퍼의 연구 결과가 말해주듯이, 대부분의 loss값 그리고 대부분의 gradient값을 차지하는 것은 easily classified negatives입니다. <br>
$ \alpha $ 를 사용해서 positive/negative examples 의 중요도를 조정할 수 있지만, easy/hard examples은 구별하지 못합니다. 

Focal loss에서는 easy examples은 down-weight를 주고, hard examples에 집중할 수 있도록 해줍니다. <br>
구체적으로는 modulating factor $ (1 - p_t)^\gamma $ 를 cross entropy에 추가를 합니다. 이때 $ \gamma \ge 0 $ 입니다. 

$$ \text{FL}(p_t) = - (1-p_t)^\gamma \log(p_t) $$

아래는 $ \gamma \in [0, 5] $ 값에 따른 그래프의 변화입니다.

![Focal Loss](images/retina-focal-loss-graph.png)

<span style="color:red">
**쉽게 설명하면!** 예측한 값이 ground-truth class에 맞을수록 영향도를 적게 주고, <br>
object를 background라고 하거나 background 를 object라고 한것처럼 개틀린것은 포커스하겠다는 뜻입니다.</span>

아래 코드를 보면.. P값이 높을수록 ground-truth class에 맞는건데, factor값이 0에 가깝습니다.<br>
예를 들어서 P값이 0.99 는 factor 값이 0.000001이고, P값이 0.01은 factor값이 0.97 입니다.

In [105]:
import numpy as np
import pandas as pd

def focal_loss(y_true, y_pred, gamma=1, visual=True):
    p = (y_true * y_pred) + ((1 - y_true) * (1 - y_pred))
    factor = (1 - p)**gamma
    loss = - (factor * np.log(factor))
    
    # Visualize    
    if visual:
        df = pd.DataFrame({'y_true': y_true,
                           'y_pred': y_pred,
                           'p': p,
                           'factor': factor,
                           'loss': loss}, 
                          columns=['y_true', 'y_pred', 'p', 'factor', 'loss'])
        display(df)
    return loss.mean()

y_true = np.array([0, 0, 0, 0, 1, 1, 1, 1], dtype=np.float64)
y_pred = np.array([0.99, 0.6, 0.3, 0.01, 0.01, 0.3, 0.6, 0.99], dtype=np.float64)
print('loss:', focal_loss(y_true, y_pred, 3))

,y_true,y_pred,p,factor,loss
0,0.0,0.99,0.01,0.970299,0.029255
1,0.0,0.60,0.40,0.216000,0.331015
2,0.0,0.30,0.70,0.027000,0.097522
3,0.0,0.01,0.99,0.000001,0.000014
4,1.0,0.01,0.01,0.970299,0.029255
5,1.0,0.30,0.30,0.343000,0.367019
6,1.0,0.60,0.60,0.064000,0.175928
7,1.0,0.99,0.99,0.000001,0.000014


loss: 0.12875271939682145
